In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

# 下面的代码试图复现malinois preprocessing

In [2]:
df = pd.read_csv('../data/Gosai_MPRA/metadata.csv', comment='#')
df

,Accession,Database,Description,Link,Usage,experiment_accession,file_accession,project,cell_type,ref_accession
0,ENCFF996ECA,ENCODE,MPRA normalized count data for OL27 in SK-N-SH...,https://www.encodeproject.org/files/ENCFF996EC...,MPRA experimental data used for Malinois train...,ENCSR623DJZ,ENCFF304FDH,OL27,SK-N-SH,ENCFF715XTT
1,ENCFF018AMJ,ENCODE,MPRA normalized count data for OL27 in A549 cells,https://www.encodeproject.org/files/ENCFF018AM...,MPRA experimental data used for Malinois train...,ENCSR729IRS,ENCFF439ZER,OL27,A549,ENCFF715XTT
2,ENCFF345ASG,ENCODE,MPRA normalized count data for OL27 in GM12878...,https://www.encodeproject.org/files/ENCFF345AS...,MPRA experimental data used for Malinois train...,ENCSR645CIP,ENCFF725MUJ,OL27,GM12878,ENCFF715XTT
3,ENCFF970OLE,ENCODE,MPRA normalized count data for OL27 in K562 cells,https://www.encodeproject.org/files/ENCFF970OL...,MPRA experimental data used for Malinois train...,ENCSR857JFG,ENCFF172YVF,OL27,K562,ENCFF715XTT
4,ENCFF318XMJ,ENCODE,MPRA normalized count data for OL27 in HepG2 c...,https://www.encodeproject.org/files/ENCFF318XM...,MPRA experimental data used for Malinois train...,ENCSR847SNQ,ENCFF224YJV,OL27,HepG2,ENCFF715XTT
5,ENCFF358MBK,ENCODE,MPRA normalized count data for OL28 in SK-N-SH...,https://www.encodeproject.org/files/ENCFF358MB...,MPRA experimental data used for Malinois train...,ENCSR833GXJ,ENCFF636IWQ,OL28,SK-N-SH,ENCFF384XZU
6,ENCFF379XWL,ENCODE,MPRA normalized count data for OL28 in HepG2 c...,https://www.encodeproject.org/files/ENCFF379XW...,MPRA experimental data used for Malinois train...,ENCSR979TSK,ENCFF847LZR,OL28,HepG2,ENCFF384XZU
7,ENCFF774CHX,ENCODE,MPRA normalized count data for OL28 in K562 cells,https://www.encodeproject.org/files/ENCFF774CH...,MPRA experimental data used for Malinois train...,ENCSR653LWA,ENCFF017YYT,OL28,K562,ENCFF384XZU
8,ENCFF138DJM,ENCODE,MPRA normalized count data for OL28 in A549 cells,https://www.encodeproject.org/files/ENCFF138DJ...,MPRA experimental data used for Malinois train...,ENCSR523VYM,ENCFF282DRV,OL28,A549,ENCFF384XZU
9,ENCFF277DDE,ENCODE,MPRA normalized count data for OL29 in A549 cells,https://www.encodeproject.org/files/ENCFF277DD...,MPRA experimental data used for Malinois train...,ENCSR273YGD,ENCFF025ATT,OL29,A549,ENCFF074MMO


In [3]:
import gzip

def read_fasta_gz_to_dict(fasta_gz_path):
    """
    从 .fasta.gz 文件读取并转换为 {id: sequence} 字典。
    """
    seq_dict = {}
    with gzip.open(fasta_gz_path, "rt") as f:
        seq_id = None
        seq_lines = []
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if seq_id is not None:
                    seq_dict[seq_id] = "".join(seq_lines)
                seq_id = line[1:]  # 去掉 '>'
                seq_lines = []
            else:
                seq_lines.append(line)
        # 处理最后一条序列
        if seq_id is not None:
            seq_dict[seq_id] = "".join(seq_lines)
    return seq_dict


def remove_key_bracket(multi_key_dict):
    expanded_dic = {}
    for compound_key, seq in multi_key_dict.items():
        # 去除外层括号（如果存在）
        if compound_key.startswith("(") and compound_key.endswith(")"):
            compound_key = compound_key[1:-1]
        # 用 ; 拆分
        keys = compound_key.split(";")
        for key in keys:
            if key in expanded_dic:
                if expanded_dic[key] != seq:
                    print(f"conflict key {key}")
                    expanded_dic[key] = None
                else: #已存在且值一致，跳过
                    print(f"conflict key {key} has same values! skip")
                    continue
            else:
                expanded_dic[key] = seq
    return expanded_dic






def merge_dicts_show_conflict(d1, d2):
    merged = {}
    for key in d1.keys() | d2.keys():  # 所有键的并集
        if key in d1 and key in d2:
            if d1[key] != d2[key]:
                print(f'conflict key {key}')
                # merged[key] = [d1[key], d2[key]]  # 不同就展示冲突
            else:
                merged[key] = d1[key]  # 相同就保留一个
        elif key in d1:
            merged[key] = d1[key]
        else:
            merged[key] = d2[key]
    return merged


In [4]:
# metadata_df = pd.read_csv('../data/Gosai_MPRA/metadata.csv')


# total_dic = {}
# ref_accessions = list(set(metadata_df['ref_accession']))

# for ref_accession in ref_accessions:
#     dic = read_fasta_gz_to_dict(f"../data/Gosai_MPRA/raw/{ref_accession}.fasta.gz")
#     dic = expand_dic(dic)
#     for key in dic:
#         if key in total_dic:
#             if total_dic[key] != dic[key]:
#                 print(f"merge conflict key '{key}' has different values!")
#                 total_dic[key] = None
#         else:
#             total_dic[key] = dic[key]

# seq_dic = total_dic
# # 暂时忽略相同ID不同序列的情况

In [5]:
def map_data_project(ol):
    ukbb_list = ['OL27', 'OL28', 'OL29', 'OL30', 'OL31', 'OL32', 'OL33']
    gtex_list = ['OL41', 'OL42', 'OL41_42', 'OL41B']
    ol15_list = ['OL15']
    if ol in ukbb_list:
        return 'UKBB'
    elif ol in gtex_list:
        return 'GTEx'
    elif ol in ol15_list:
        return 'OL15'
    else:
        raise ValueError(f'Unknown OL: {ol}')

In [6]:
from collections import Counter

metadata_df = pd.read_csv('../data/Gosai_MPRA/metadata.csv')
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']

dfss = []
for cell_type in cell_types:
    dfs = []
    for i, row in metadata_df.iterrows():
        if row['cell_type'] == cell_type:
            file_accession = row['file_accession']
            print(file_accession)

            ref_accession = row['ref_accession']
            if ';' in ref_accession:
                ref1, ref2 = ref_accession.split(';')
                dic1 = read_fasta_gz_to_dict(f"../data/Gosai_MPRA/raw/{ref1}.fasta.gz")
                dic2 = read_fasta_gz_to_dict(f"../data/Gosai_MPRA/raw/{ref2}.fasta.gz")
                dic1 = remove_key_bracket(dic1)
                dic2 = remove_key_bracket(dic2)
                dic = merge_dicts_show_conflict(dic1, dic2)
            
            else:
                dic = read_fasta_gz_to_dict(f"../data/Gosai_MPRA/raw/{ref_accession}.fasta.gz")
                dic = remove_key_bracket(dic)

            df = pd.read_csv(f"../data/Gosai_MPRA/raw/{file_accession}.tsv", sep='\t', low_memory=False)
            df['OL'] = row['project']
            df['data_project'] = map_data_project(row['project'])
            df['seq'] = df['ID'].map(dic)
            
            dfs.append(df)
    dfs = pd.concat(dfs).reset_index(drop=True)
    dfss.append(dfs)

ENCFF172YVF


ENCFF017YYT
ENCFF889WPF
ENCFF612YRJ
ENCFF842RUM
ENCFF995HVI
ENCFF744OCG
ENCFF141ZOX
ENCFF090LOS
ENCFF224YJV
ENCFF847LZR
ENCFF866FZN
ENCFF848GPT
ENCFF909YEL
ENCFF112XKL
ENCFF983QNK
ENCFF734CRQ
ENCFF539GVZ
ENCFF521ALH
ENCFF304FDH
ENCFF636IWQ
ENCFF329BLH
ENCFF480BKB
ENCFF329RAT
ENCFF885GWB
ENCFF382RYO
ENCFF094EUR
ENCFF521IVN
ENCFF209EYZ
ENCFF945UBA
ENCFF966ZKK
ENCFF439ZER
ENCFF282DRV
ENCFF025ATT
ENCFF983DIE
ENCFF772GNQ
ENCFF978EGA
ENCFF742UTE


In [7]:
def filter_seq_is_none(df):
    cond = (df['seq'].notna())
    df_filtered = df[cond].copy()
    return df_filtered



def filter_nonnatural_oligos(df):
    cond = (df['chr'].notna())
    df_filtered = df[cond].copy()
    return df_filtered



def filter_plasmid_and_rna_count(df):
    # DNA count >= 20, RNA_count > 0
    DNA_cond_1 = (df['DNA_mean'] >= 5) & (df['data_project']=='OL15') # OL15 4 replicates
    DNA_cond_2 = (df['DNA_mean'] >= 4) & (df['data_project']!='OL15') # other 5 replicates
    RNA_cond = (df['exp_mean'] > 0)
    cond = (DNA_cond_1 | DNA_cond_2) & RNA_cond
    df_filtered = df[cond].copy()
    return df_filtered



def filter_log2fc_6std(df):
    std = df['log2FoldChange'].std()
    mean = df['log2FoldChange'].mean()
    cond = (df['log2FoldChange'] >= (mean - 6 * std))
    df_filtered = df[cond].copy()
    return df_filtered



def merge_ukbb_gtex_ol15(df):
    # 定义优先级映射
    priority_map = {'UKBB': 3, 'GTEx': 2, 'OL15': 1}
    # 给 dataframe 新增一列 priority 便于后续比较
    df['priority'] = df['data_project'].map(priority_map)
    if df['priority'].isna().any():
        print('存在 priority 为空的行!!')
    
    # 构造聚合字典：log2FoldChange 做 mean，其余列都用 'first'
    # 如果你的表有很多列，只关心部分字段，也可以只在聚合字典中声明关心的列。
    agg_dict = {}
    for col in df.columns:
        if col == 'log2FoldChange':
            agg_dict[col] = 'mean'
        else:
            # 取第一行，如果同一个 (ID, data_project) 下这些列确实没有冲突
            agg_dict[col] = 'first'

    # 一次分组：对 ID + data_project 分组
    df_agg = df.groupby(['seq', 'data_project'], as_index=False).agg(agg_dict)

    # 对聚合完的数据按优先级降序排；同一个 ID 下，UKBB(3) > GTEx(2) > OL15(1)
    df_agg.sort_values(['seq', 'priority'], ascending=[True, False], inplace=True)

    # 同一个 ID 只保留优先级最高的项目那条记录
    df_final = df_agg.drop_duplicates(subset='seq', keep='first').copy()

    # 不需要 priority 列了，可以删除
    df_final.drop(columns='priority', inplace=True)
    
    return df_final

In [12]:
merged_df = pd.DataFrame(columns=dfss[0].columns)

for i in range(5):
    df_i = dfss[i]
    print(len(df_i))
    df_i = filter_nonnatural_oligos(df_i)
    print(len(df_i))
    df_i = filter_seq_is_none(df_i)
    print(len(df_i))
    df_i = filter_plasmid_and_rna_count(df_i)
    print(len(df_i))
    df_i = merge_ukbb_gtex_ol15(df_i)
    print(len(df_i))
    df_i = filter_log2fc_6std(df_i)
    print(len(df_i))

    merged_df = pd.merge(merged_df, df_i, on=['seq'], how='outer', suffixes=('', f'_{i}'))
    print('')

944435
929435
929429
908354
797664
797664

945301
930301
930295
909183
797959
797955

945301
930301
930295
908141
797019
797014

494592
494592
494590
481313
467896
467896

420718
420718
420714
413632
338286
338233



In [13]:
merged_df

,ID,SNP,chr,pos,ref_allele,alt_allele,allele,window,strand,project,haplotype,ctrl_exp,DNA_mean,ctrl_mean,exp_mean,log2FoldChange,lfcSE,stat,pvalue,padj,OL,data_project,seq,start,stop,ID_0,SNP_0,chr_0,pos_0,ref_allele_0,alt_allele_0,allele_0,window_0,strand_0,project_0,haplotype_0,ctrl_exp_0,DNA_mean_0,ctrl_mean_0,exp_mean_0,log2FoldChange_0,lfcSE_0,stat_0,pvalue_0,padj_0,OL_0,data_project_0,start_0,stop_0,ID_1,SNP_1,chr_1,pos_1,ref_allele_1,alt_allele_1,allele_1,window_1,strand_1,project_1,haplotype_1,ctrl_exp_1,DNA_mean_1,ctrl_mean_1,exp_mean_1,log2FoldChange_1,lfcSE_1,stat_1,pvalue_1,padj_1,OL_1,data_project_1,start_1,stop_1,ID_2,SNP_2,chr_2,pos_2,ref_allele_2,alt_allele_2,allele_2,window_2,strand_2,project_2,haplotype_2,ctrl_exp_2,DNA_mean_2,ctrl_mean_2,exp_mean_2,log2FoldChange_2,lfcSE_2,stat_2,pvalue_2,padj_2,OL_2,data_project_2,start_2,stop_2,ID_3,SNP_3,chr_3,pos_3,ref_allele_3,alt_allele_3,allele_3,window_3,strand_3,project_3,haplotype_3,ctrl_exp_3,DNA_mean_3,ctrl_mean_3,exp_mean_3,log2FoldChange_3,lfcSE_3,stat_3,pvalue_3,padj_3,OL_3,data_project_3,ID_4,SNP_4,chr_4,pos_4,ref_allele_4,alt_allele_4,allele_4,window_4,strand_4,project_4,haplotype_4,ctrl_exp_4,DNA_mean_4,ctrl_mean_4,exp_mean_4,log2FoldChange_4,lfcSE_4,stat_4,pvalue_4,padj_4,OL_4,data_project_4
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,NaN,NaN,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL41_42,467.400,334.316,434.641,0.379,0.252,1.504,0.133,0.264,OL41_42,GTEx,NaN,NaN,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL42,533.800,408.445,395.638,-0.049,0.143,-0.341,0.733,0.799,OL42,GTEx,NaN,NaN,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL42,533.800,408.445,378.529,-0.119,0.370,-0.321,0.748,0.845,OL42,GTEx,NaN,NaN,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL42,533.800,408.445,333.648,-0.291,0.159,-1.834,0.067,0.127,OL42,GTEx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,NaN,NaN,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL41_42,25.800,18.766,14.227,-0.401,1.720,-0.233,0.816,0.903,OL41_42,GTEx,NaN,NaN,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL42,21.800,17.099,19.904,0.215,1.005,0.214,0.830,0.876,OL42,GTEx,NaN,NaN,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL42,21.800,17.099,3.438,-2.281,2.928,-0.779,0.436,0.594,OL42,GTEx,NaN,NaN,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL42,21.800,17.099,13.619,-0.329,1.190,-0.276,0.782,0.858,OL42,GTEx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGAATGTCAGTT...,NaN,NaN,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,G,alt,center,fwd,BMI,ref,OL29,12.400,8.856,17.171,0.956,0.527,1.812,0.070,0.107,OL29,UKBB,NaN,NaN,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,G,alt,center,fwd,BMI,ref,OL29,12.400,8.856,10.882,0.297,1.291,0.230,0.818,0.877,OL29,UKBB,NaN,NaN,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,G,alt,center,fwd,BMI,ref,OL29,12.400,8.856,5.091,-0.801,2.904,-0.276,0.783,0.879,OL29,UKBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,G,alt,center,fwd,BMI,ref,OL29,12.400,8.856,22.818,1.360,0.578,2.354,0.019,0.054,OL29,UKBB
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,NaN,NaN,19:929473

In [15]:
for col in ['ID', 'chr', 'pos', 'ref_allele', 'alt_allele', 'allele', 'OL', 'data_project']:
    cols = [f'{col}_{i}' for i in range(5)]
    merged_df[f'{col}_merged'] = merged_df[cols].bfill(axis=1).iloc[:, 0]

/tmp/ipykernel_82026/3077366796.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df[f'{col}_merged'] = merged_df[cols].bfill(axis=1).iloc[:, 0]
/tmp/ipykernel_82026/3077366796.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df[f'{col}_merged'] = merged_df[cols].bfill(axis=1).iloc[:, 0]
/tmp/ipykernel_82026/3077366796.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.

In [16]:
merged_df

,ID,SNP,chr,pos,ref_allele,alt_allele,allele,window,strand,project,haplotype,ctrl_exp,DNA_mean,ctrl_mean,exp_mean,log2FoldChange,lfcSE,stat,pvalue,padj,OL,data_project,seq,start,stop,ID_0,SNP_0,chr_0,pos_0,ref_allele_0,alt_allele_0,allele_0,window_0,strand_0,project_0,haplotype_0,ctrl_exp_0,DNA_mean_0,ctrl_mean_0,exp_mean_0,log2FoldChange_0,lfcSE_0,stat_0,pvalue_0,padj_0,OL_0,data_project_0,start_0,stop_0,ID_1,SNP_1,chr_1,pos_1,ref_allele_1,alt_allele_1,allele_1,window_1,strand_1,project_1,haplotype_1,ctrl_exp_1,DNA_mean_1,ctrl_mean_1,exp_mean_1,log2FoldChange_1,lfcSE_1,stat_1,pvalue_1,padj_1,OL_1,data_project_1,start_1,stop_1,ID_2,SNP_2,chr_2,pos_2,ref_allele_2,alt_allele_2,allele_2,window_2,strand_2,project_2,haplotype_2,ctrl_exp_2,DNA_mean_2,ctrl_mean_2,exp_mean_2,log2FoldChange_2,lfcSE_2,stat_2,pvalue_2,padj_2,OL_2,data_project_2,start_2,stop_2,ID_3,SNP_3,chr_3,pos_3,ref_allele_3,alt_allele_3,allele_3,window_3,strand_3,project_3,haplotype_3,ctrl_exp_3,DNA_mean_3,ctrl_mean_3,exp_mean_3,log2FoldChange_3,lfcSE_3,stat_3,pvalue_3,padj_3,OL_3,data_project_3,ID_4,SNP_4,chr_4,pos_4,ref_allele_4,alt_allele_4,allele_4,window_4,strand_4,project_4,haplotype_4,ctrl_exp_4,DNA_mean_4,ctrl_mean_4,exp_mean_4,log2FoldChange_4,lfcSE_4,stat_4,pvalue_4,padj_4,OL_4,data_project_4,ID_merged,chr_merged,pos_merged,ref_allele_merged,alt_allele_merged,allele_merged,OL_merged,data_project_merged
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,NaN,NaN,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL41_42,467.400,334.316,434.641,0.379,0.252,1.504,0.133,0.264,OL41_42,GTEx,NaN,NaN,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL42,533.800,408.445,395.638,-0.049,0.143,-0.341,0.733,0.799,OL42,GTEx,NaN,NaN,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL42,533.800,408.445,378.529,-0.119,0.370,-0.321,0.748,0.845,OL42,GTEx,NaN,NaN,1:161114366:A:T:R:wC,1:161114366:A:T,1,161114366.000,A,T,ref,center,fwd,GTEx,ref,OL42,533.800,408.445,333.648,-0.291,0.159,-1.834,0.067,0.127,OL42,GTEx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:161114366:A:T:R:wC,1,161114366.000,A,T,ref,OL41_42,GTEx
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,NaN,NaN,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL41_42,25.800,18.766,14.227,-0.401,1.720,-0.233,0.816,0.903,OL41_42,GTEx,NaN,NaN,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL42,21.800,17.099,19.904,0.215,1.005,0.214,0.830,0.876,OL42,GTEx,NaN,NaN,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL42,21.800,17.099,3.438,-2.281,2.928,-0.779,0.436,0.594,OL42,GTEx,NaN,NaN,1:161114366:A:T:A:wC,1:161114366:A:T,1,161114366.000,A,T,alt,center,fwd,GTEx,ref,OL42,21.800,17.099,13.619,-0.329,1.190,-0.276,0.782,0.858,OL42,GTEx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:161114366:A:T:A:wC,1,161114366.000,A,T,alt,OL41_42,GTEx
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGAATGTCAGTT...,NaN,NaN,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,G,alt,center,fwd,BMI,ref,OL29,12.400,8.856,17.171,0.956,0.527,1.812,0.070,0.107,OL29,UKBB,NaN,NaN,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,G,alt,center,fwd,BMI,ref,OL29,12.400,8.856,10.882,0.297,1.291,0.230,0.818,0.877,OL29,UKBB,NaN,NaN,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,G,alt,center,fwd,BMI,ref,OL29,12.400,8.856,5.091,-0.801,2.904,-0.276,0.783,0.879,OL29,UKBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:89640175:A:G:A:wC,7:89640175:A:G,7,89640175.000,A,G,alt,center,fwd,BM

In [17]:
cols = ['seq'] + [c for c in merged_df.columns if c.endswith('_merged') or c.startswith('log2FoldChange_')]
merged_df = merged_df[cols]

merged_df = merged_df.rename(columns={
    'log2FoldChange_0': 'K562',
    'log2FoldChange_1': 'HepG2',
    'log2FoldChange_2': 'SK-N-SH',
    'log2FoldChange_3': 'A549',
    'log2FoldChange_4': 'HCT116',
})

merged_df = merged_df.rename(columns={c: c.replace('_merged', '') for c in merged_df.columns if c.endswith('_merged')})
merged_df = merged_df[['seq', 'ID', 'chr', 'pos', 'ref_allele', 'alt_allele', 'allele', 'OL', 'data_project', 'K562', 'HepG2', 'SK-N-SH', 'A549', 'HCT116']]
merged_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:R:wC,1,161114366.000,A,T,ref,OL41_42,GTEx,0.379,-0.049,-0.119,-0.291,NaN
1,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:A:wC,1,161114366.000,A,T,alt,OL41_42,GTEx,-0.401,0.215,-2.281,-0.329,NaN
2,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGAATGTCAGTT...,7:89640175:A:G:A:wC,7,89640175.000,A,G,alt,OL29,UKBB,0.956,0.297,-0.801,NaN,1.360
3,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,19:9294737:G:A:A:wC,19,9294737.000,G,A,alt,OL41_42,GTEx,-0.101,0.468,-0.154,-0.168,NaN
4,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,19:9294737:G:A:R:wC,19,9294737.000,G,A,ref,OL41_42,GTEx,0.154,0.352,-0.016,0.116,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799896,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCG...,6:153341374:C:T:R:wC,6,153341374.000,C,T,ref,OL41_42,GTEx,0.291,-0.115,0.004,-0.402,NaN
799897,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCG...,6:153341374:C:T:A:wC,6,153341374.000,C,T,alt,OL41_42,GTEx,-0.085,NaN,-0.273,NaN,NaN
799898,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACGGAGTCTTG...,2:181602082:C:T:R:wC,2,181602082.000,C,T,ref,OL41_42,GTEx,2.276,2.067,1.982,2.346,NaN
799899,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACGGAGTCTTG...,2:181602082:C:T:A:wC,2,181602082.000,C,T,alt,OL41_42,GTEx,-0.016,0.614,0.449,0.308,NaN


In [18]:
merged_df['chr'] = 'chr' + merged_df['chr'].astype(str)
merged_df['pos'] = pd.to_numeric(merged_df['pos'], errors='coerce').astype('Int64')

merged_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:R:wC,chr1,161114366,A,T,ref,OL41_42,GTEx,0.379,-0.049,-0.119,-0.291,NaN
1,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGGCTGGGTG...,1:161114366:A:T:A:wC,chr1,161114366,A,T,alt,OL41_42,GTEx,-0.401,0.215,-2.281,-0.329,NaN
2,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGAATGTCAGTT...,7:89640175:A:G:A:wC,chr7,89640175,A,G,alt,OL29,UKBB,0.956,0.297,-0.801,NaN,1.360
3,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,19:9294737:G:A:A:wC,chr19,9294737,G,A,alt,OL41_42,GTEx,-0.101,0.468,-0.154,-0.168,NaN
4,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,19:9294737:G:A:R:wC,chr19,9294737,G,A,ref,OL41_42,GTEx,0.154,0.352,-0.016,0.116,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799896,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCG...,6:153341374:C:T:R:wC,chr6,153341374,C,T,ref,OL41_42,GTEx,0.291,-0.115,0.004,-0.402,NaN
799897,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCG...,6:153341374:C:T:A:wC,chr6,153341374,C,T,alt,OL41_42,GTEx,-0.085,NaN,-0.273,NaN,NaN
799898,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACGGAGTCTTG...,2:181602082:C:T:R:wC,chr2,181602082,C,T,ref,OL41_42,GTEx,2.276,2.067,1.982,2.346,NaN
799899,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACGGAGTCTTG...,2:181602082:C:T:A:wC,chr2,181602082,C,T,alt,OL41_42,GTEx,-0.016,0.614,0.449,0.308,NaN


In [19]:
merged_df = merged_df.sort_values(by=['chr', 'pos']).reset_index(drop=True)
merged_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:A:wC,chr1,14677,G,A,alt,OL41_42,GTEx,0.554,0.416,0.442,-0.288,NaN
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:R:wC,chr1,14677,G,A,ref,OL41_42,GTEx,-0.704,0.668,0.539,0.159,NaN
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:A:wC,chr1,63697,T,C,alt,OL41_42,GTEx,-0.659,1.152,0.205,1.051,NaN
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:R:wC,chr1,63697,T,C,ref,OL41_42,GTEx,-0.391,1.270,0.752,1.100,NaN
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,1:64764:C:T:R:wC,chr1,64764,C,T,ref,OL41_42,GTEx,-0.007,-0.211,0.499,0.258,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799896,CTGCATCTGTGCAGCCTGGCAGCGGCGGCGCTGCGCTGTGACATTT...,Y:15815629:NA:NA,chrY,15815629,NaN,NaN,NaN,OL15,OL15,1.381,1.846,1.702,NaN,NaN
799897,GCACTTCACATTTGTTTTTAGGGTTACATAGTCTACTCTGTATCCT...,Y:21740684:NA:NA,chrY,21740684,NaN,NaN,NaN,OL15,OL15,-0.306,-0.107,-0.080,NaN,NaN
799898,TGTTCAAGTGGCCACAGGGTTACTTGCTTTAGCATGGCTCCTTGGC...,Y:21740884:NA:NA,chrY,21740884,NaN,NaN,NaN,OL15,OL15,0.363,0.261,0.060,NaN,NaN
799899,TAATTAGTTGGGAAGGTTCAGGTTCTGGGACATCCGTTGCTATTTC...,Y:21741084:NA:NA,chrY,21741084,NaN,NaN,NaN,OL15,OL15,0.395,0.378,-0.030,NaN,NaN


In [20]:
merged_df.to_csv('../data/Gosai_MPRA/Gosai_MPRA_799901.csv', index=False)

In [21]:
my_processed_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_799901.csv')
my_processed_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:A:wC,chr1,14677,G,A,alt,OL41_42,GTEx,0.554,0.416,0.442,-0.288,NaN
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:R:wC,chr1,14677,G,A,ref,OL41_42,GTEx,-0.704,0.668,0.539,0.159,NaN
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:A:wC,chr1,63697,T,C,alt,OL41_42,GTEx,-0.659,1.152,0.205,1.051,NaN
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:R:wC,chr1,63697,T,C,ref,OL41_42,GTEx,-0.391,1.270,0.752,1.100,NaN
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,1:64764:C:T:R:wC,chr1,64764,C,T,ref,OL41_42,GTEx,-0.007,-0.211,0.499,0.258,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799896,CTGCATCTGTGCAGCCTGGCAGCGGCGGCGCTGCGCTGTGACATTT...,Y:15815629:NA:NA,chrY,15815629,NaN,NaN,NaN,OL15,OL15,1.381,1.846,1.702,NaN,NaN
799897,GCACTTCACATTTGTTTTTAGGGTTACATAGTCTACTCTGTATCCT...,Y:21740684:NA:NA,chrY,21740684,NaN,NaN,NaN,OL15,OL15,-0.306,-0.107,-0.080,NaN,NaN
799898,TGTTCAAGTGGCCACAGGGTTACTTGCTTTAGCATGGCTCCTTGGC...,Y:21740884:NA:NA,chrY,21740884,NaN,NaN,NaN,OL15,OL15,0.363,0.261,0.060,NaN,NaN
799899,TAATTAGTTGGGAAGGTTCAGGTTCTGGGACATCCGTTGCTATTTC...,Y:21741084:NA:NA,chrY,21741084,NaN,NaN,NaN,OL15,OL15,0.395,0.378,-0.030,NaN,NaN


In [22]:
my_processed_df['data_project'].value_counts()

data_project
GTEx    447167
UKBB    338731
OL15     14003
Name: count, dtype: int64

In [24]:
my_processed_df[my_processed_df['seq'].duplicated(keep=False)]

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116


In [25]:
my_processed_df[my_processed_df['ID'].duplicated(keep=False)]

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116
396902,CCTCCTCCAGCAGGACTCCCACCTAGCCTGAAGGTCGGATGGATCT...,19:54676763:C:T:R:wC:Alt_19-54676814-A,chr19,54676763,C,T,ref,OL32,UKBB,0.088,0.522,-0.231,NaN,0.025
396904,CCTCCTCCAGCAGGACTCCCACCTAGCCTGAAGGTCGGATGGATCT...,19:54676763:C:T:A:wC:Alt_19-54676814-A,chr19,54676763,C,T,alt,OL32,UKBB,0.117,0.528,0.016,NaN,0.187
396905,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...,19:54676763:C:T:R:wL:Alt_19-54676814-A,chr19,54676763,C,T,ref,OL41_42,GTEx,0.135,0.996,-1.089,2.453,NaN
396906,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...,19:54676763:C:T:R:wL:Alt_19-54676814-R,chr19,54676763,C,T,ref,OL32,UKBB,0.181,0.643,-1.299,NaN,-0.541
396907,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...,19:54676763:C:T:R:wL:Alt_19-54676814-R,chr19,54676763,C,T,ref,OL41_42,GTEx,0.304,0.698,-0.943,2.451,NaN
396909,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...,19:54676763:C:T:R:wL:Alt_19-54676814-A,chr19,54676763,C,T,ref,OL32,UKBB,0.035,0.694,-1.011,NaN,-0.061
396910,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...,19:54676763:C:T:A:wL:Alt_19-54676814-A,chr19,54676763,C,T,alt,OL41_42,GTEx,0.410,1.034,-0.004,2.773,NaN
396911,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...,19:54676763:C:T:A:wL:Alt_19-54676814-R,chr19,54676763,C,T,alt,OL32,UKBB,-0.018,0.735,-0.947,NaN,0.345
396912,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...,19:54676763:C:T:A:wL:Alt_19-54676814-R,chr19,54676763,C,T,alt,OL41_42,GTEx,0.131,0.739,-1.027,2.643,NaN
396914,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...,19:54676763:C:T:A:wL:Alt_19-54676814-A,chr19,54676763,C,T,alt,OL32,UKBB,-0.001,0.527,-1.684,NaN,0.033


In [26]:
original_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA.csv')
original_df

# original_df['long_ID'] = original_df['ID'].apply(lambda x: x.split(',')[-1])
# original_df['ID'] = original_df['long_ID'].map(get_core_id)
# original_df

/tmp/ipykernel_82026/1618383038.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA.csv')


,ID,chr,data_project,OL,K562,HepG2,SK-N-SH,seq
0,7:70038969:G:T:A:wC,chr7,UKBB,29,0.061,0.234,0.047,CCTGGTCTTTCTTGCTAAATAAACATATCGTGCATCATCCAGATCT...
1,1:192696196:C:T:A:wC,chr1,UKBB,33,0.380,0.005,-0.244,CATAAAGATGAGGCTTGGCAAAGAACATCTCTCGGTGCCTCCCATT...
2,1:211209457:C:T:A:wC,chr1,UKBB,33,0.037,0.385,-0.005,CATAAAGCCAATCACTGAGATGACAAGTACTGCCAGGAAAGAAGGC...
3,15:89574440:GT:G:A:wC,chr15,UKBB,33,4.465,4.107,2.870,CATAAAGGCAGTGTAGACCCAAACAGTGAGCAGTAGCAAGATTTAT...
4,15:89574440:GT:G:R:wC,chr15,UKBB,33,4.509,4.116,3.040,CATAAAGGCAGTGTAGACCCAAACAGTGAGCAGTAGCAAGATTTAT...
...,...,...,...,...,...,...,...,...
798059,4:44680358:NA:NA,chr4,CRE,15,7.444,5.344,6.585,CAGTAGTAAGAAAGAGACAATGCAAAGGAATTGGCACAGCACTCAG...
798060,18:9125893:NA:NA,chr18,CRE,15,-0.205,-0.157,-0.209,CAGTACTGCTGGCCCCAGAAAAGCCCCTCTCCTTATACCCTAGGCC...
798061,12:33905808:NA:NA,chr12,CRE,15,1.218,0.614,0.570,CAGTACCTTGTCCCCACTTCCCATTTGGCCTCTGGCAGAGGAGGAG...
798062,3:128145854:NA:NA,chr3,CRE,15,-0.222,-0.339,-0.818,CAGTACACCCCAGCTTCCAAAGGCCTTCTGTGACAAAGAGAGACTA...


In [27]:
original_df['data_project'].value_counts()

data_project
GTEX    445580
UKBB    338398
CRE      14086
Name: count, dtype: int64

In [28]:
original_df[original_df['seq'].duplicated(keep=False)]

,ID,chr,data_project,OL,K562,HepG2,SK-N-SH,seq


In [29]:
original_df[original_df['ID'].duplicated(keep=False)]

,ID,chr,data_project,OL,K562,HepG2,SK-N-SH,seq
54562,19:54676763:C:T:R:wL:Alt_19-54676814-A,chr19,UKBB,32,0.019,0.702,-1.027,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...
54564,19:54676763:C:T:A:wL:Alt_19-54676814-A,chr19,UKBB,32,0.007,0.518,-1.589,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...
64731,19:54676763:C:T:A:wC:Alt_19-54676814-A,chr19,UKBB,32,0.118,0.572,0.028,CCTCCTCCAGCAGGACTCCCACCTAGCCTGAAGGTCGGATGGATCT...
64749,19:54676763:C:T:R:wC:Alt_19-54676814-A,chr19,UKBB,32,0.086,0.553,-0.242,CCTCCTCCAGCAGGACTCCCACCTAGCCTGAAGGTCGGATGGATCT...
651490,19:54676763:C:T:R:wC:Alt_19-54676814-A,chr19,GTEX,42,-0.386,0.630,-0.031,TCCTCCTCCAGCAGGACTCCCACCTAGCCTGAAGGTCGGATGGATC...
651491,19:54676763:C:T:A:wC:Alt_19-54676814-A,chr19,GTEX,42,-0.235,0.798,0.220,TCCTCCTCCAGCAGGACTCCCACCTAGCCTGAAGGTCGGATGGATC...
696845,19:54676763:C:T:A:wL:Alt_19-54676814-A,chr19,GTEX,42,0.371,1.033,-0.001,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...
696852,19:54676763:C:T:R:wL:Alt_19-54676814-A,chr19,GTEX,42,0.097,0.995,-1.084,CTTCTCCTGGCATTCCCTACCTCCTCTGGCCTCCCGGGGGGCCAGC...


In [34]:
merged_df = pd.merge(original_df, my_processed_df, on='seq', how='left')
merged_df

,ID_x,chr_x,data_project_x,OL_x,K562_x,HepG2_x,SK-N-SH_x,seq,ID_y,chr_y,pos,ref_allele,alt_allele,allele,OL_y,data_project_y,K562_y,HepG2_y,SK-N-SH_y,A549,HCT116
0,7:70038969:G:T:A:wC,chr7,UKBB,29,0.061,0.234,0.047,CCTGGTCTTTCTTGCTAAATAAACATATCGTGCATCATCCAGATCT...,7:70038969:G:T:A:wC,chr7,70038969.000,G,T,alt,OL29,UKBB,0.060,0.239,0.023,NaN,0.010
1,1:192696196:C:T:A:wC,chr1,UKBB,33,0.380,0.005,-0.244,CATAAAGATGAGGCTTGGCAAAGAACATCTCTCGGTGCCTCCCATT...,1:192696196:C:T:A:wC,chr1,192696196.000,C,T,alt,OL33,UKBB,0.374,0.034,-0.242,NaN,-0.024
2,1:211209457:C:T:A:wC,chr1,UKBB,33,0.037,0.385,-0.005,CATAAAGCCAATCACTGAGATGACAAGTACTGCCAGGAAAGAAGGC...,1:211209457:C:T:A:wC,chr1,211209457.000,C,T,alt,OL33,UKBB,0.037,0.393,-0.005,NaN,-0.028
3,15:89574440:GT:G:A:wC,chr15,UKBB,33,4.465,4.107,2.870,CATAAAGGCAGTGTAGACCCAAACAGTGAGCAGTAGCAAGATTTAT...,15:89574440:GT:G:A:wC,chr15,89574440.000,GT,G,alt,OL33,UKBB,4.467,4.116,2.867,NaN,3.531
4,15:89574440:GT:G:R:wC,chr15,UKBB,33,4.509,4.116,3.040,CATAAAGGCAGTGTAGACCCAAACAGTGAGCAGTAGCAAGATTTAT...,15:89574440:GT:G:R:wC,chr15,89574440.000,GT,G,ref,OL33,UKBB,4.506,4.122,3.039,NaN,3.621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798059,4:44680358:NA:NA,chr4,CRE,15,7.444,5.344,6.585,CAGTAGTAAGAAAGAGACAATGCAAAGGAATTGGCACAGCACTCAG...,4:44680358:NA:NA,chr4,44680358.000,NaN,NaN,NaN,OL15,OL15,7.444,5.345,6.585,NaN,NaN
798060,18:9125893:NA:NA,chr18,CRE,15,-0.205,-0.157,-0.209,CAGTACTGCTGGCCCCAGAAAAGCCCCTCTCCTTATACCCTAGGCC...,18:9125893:NA:NA,chr18,9125893.000,NaN,NaN,NaN,OL15,OL15,-0.205,-0.156,-0.210,NaN,NaN
798061,12:33905808:NA:NA,chr12,CRE,15,1.218,0.614,0.570,CAGTACCTTGTCCCCACTTCCCATTTGGCCTCTGGCAGAGGAGGAG...,12:33905808:NA:NA,chr12,33905808.000,NaN,NaN,NaN,OL15,OL15,1.218,0.614,0.570,NaN,NaN
798062,3:128145854:NA:NA,chr3,CRE,15,-0.222,-0.339,-0.818,CAGTACACCCCAGCTTCCAAAGGCCTTCTGTGACAAAGAGAGACTA...,3:128145854:NA:NA,chr3,128145854.000,NaN,NaN,NaN,OL15,OL15,-0.224,-0.338,-0.817,NaN,NaN


In [36]:
merged_df.isna().sum()

ID_x                   0
chr_x                  0
data_project_x         0
OL_x                   0
K562_x                 0
HepG2_x                0
SK-N-SH_x              0
seq                    0
ID_y                1613
chr_y               1613
pos                 1613
ref_allele         15991
alt_allele         15991
allele             15991
OL_y                1613
data_project_y      1613
K562_y              2449
HepG2_y             2414
SK-N-SH_y           2420
A549              331814
HCT116            461141
dtype: int64

In [37]:
cols = ['seq'] + [c for c in merged_df.columns if c.endswith('_x')] + ['HCT116', 'A549']
merged_df = merged_df[cols]
merged_df.rename(columns={c: c.replace('_x', '') for c in merged_df.columns}, inplace=True)
merged_df

/tmp/ipykernel_82026/3496631077.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.rename(columns={c: c.replace('_x', '') for c in merged_df.columns}, inplace=True)


,seq,ID,chr,data_project,OL,K562,HepG2,SK-N-SH,HCT116,A549
0,CCTGGTCTTTCTTGCTAAATAAACATATCGTGCATCATCCAGATCT...,7:70038969:G:T:A:wC,chr7,UKBB,29,0.061,0.234,0.047,0.010,NaN
1,CATAAAGATGAGGCTTGGCAAAGAACATCTCTCGGTGCCTCCCATT...,1:192696196:C:T:A:wC,chr1,UKBB,33,0.380,0.005,-0.244,-0.024,NaN
2,CATAAAGCCAATCACTGAGATGACAAGTACTGCCAGGAAAGAAGGC...,1:211209457:C:T:A:wC,chr1,UKBB,33,0.037,0.385,-0.005,-0.028,NaN
3,CATAAAGGCAGTGTAGACCCAAACAGTGAGCAGTAGCAAGATTTAT...,15:89574440:GT:G:A:wC,chr15,UKBB,33,4.465,4.107,2.870,3.531,NaN
4,CATAAAGGCAGTGTAGACCCAAACAGTGAGCAGTAGCAAGATTTAT...,15:89574440:GT:G:R:wC,chr15,UKBB,33,4.509,4.116,3.040,3.621,NaN
...,...,...,...,...,...,...,...,...,...,...
798059,CAGTAGTAAGAAAGAGACAATGCAAAGGAATTGGCACAGCACTCAG...,4:44680358:NA:NA,chr4,CRE,15,7.444,5.344,6.585,NaN,NaN
798060,CAGTACTGCTGGCCCCAGAAAAGCCCCTCTCCTTATACCCTAGGCC...,18:9125893:NA:NA,chr18,CRE,15,-0.205,-0.157,-0.209,NaN,NaN
798061,CAGTACCTTGTCCCCACTTCCCATTTGGCCTCTGGCAGAGGAGGAG...,12:33905808:NA:NA,chr12,CRE,15,1.218,0.614,0.570,NaN,NaN
798062,CAGTACACCCCAGCTTCCAAAGGCCTTCTGTGACAAAGAGAGACTA...,3:128145854:NA:NA,chr3,CRE,15,-0.222,-0.339,-0.818,NaN,NaN


In [38]:
merged_df.to_csv('../data/Gosai_MPRA/Gosai_MPRA_merged_5_cell_types.csv', index=False)

In [39]:
merged_df = merged_df[merged_df['seq'].str.len() == 200].reset_index(drop=True)
merged_df

,seq,ID,chr,data_project,OL,K562,HepG2,SK-N-SH,HCT116,A549
0,CCTGGTCTTTCTTGCTAAATAAACATATCGTGCATCATCCAGATCT...,7:70038969:G:T:A:wC,chr7,UKBB,29,0.061,0.234,0.047,0.010,NaN
1,CATAAAGATGAGGCTTGGCAAAGAACATCTCTCGGTGCCTCCCATT...,1:192696196:C:T:A:wC,chr1,UKBB,33,0.380,0.005,-0.244,-0.024,NaN
2,CATAAAGCCAATCACTGAGATGACAAGTACTGCCAGGAAAGAAGGC...,1:211209457:C:T:A:wC,chr1,UKBB,33,0.037,0.385,-0.005,-0.028,NaN
3,CATAAAGGCAGTGTAGACCCAAACAGTGAGCAGTAGCAAGATTTAT...,15:89574440:GT:G:R:wC,chr15,UKBB,33,4.509,4.116,3.040,3.621,NaN
4,CATAAAGGGCTGAACATGCTGTTGAAAAAATGTAGATATAAAAGTT...,12:63513920:G:A:A:wC,chr12,UKBB,32,1.617,1.423,1.336,0.512,NaN
...,...,...,...,...,...,...,...,...,...,...
763679,CAGTAGTAAGAAAGAGACAATGCAAAGGAATTGGCACAGCACTCAG...,4:44680358:NA:NA,chr4,CRE,15,7.444,5.344,6.585,NaN,NaN
763680,CAGTACTGCTGGCCCCAGAAAAGCCCCTCTCCTTATACCCTAGGCC...,18:9125893:NA:NA,chr18,CRE,15,-0.205,-0.157,-0.209,NaN,NaN
763681,CAGTACCTTGTCCCCACTTCCCATTTGGCCTCTGGCAGAGGAGGAG...,12:33905808:NA:NA,chr12,CRE,15,1.218,0.614,0.570,NaN,NaN
763682,CAGTACACCCCAGCTTCCAAAGGCCTTCTGTGACAAAGAGAGACTA...,3:128145854:NA:NA,chr3,CRE,15,-0.222,-0.339,-0.818,NaN,NaN


In [40]:
merged_df.to_csv('../data/Gosai_MPRA/Gosai_MPRA_merged_5_cell_types_len_200.csv', index=False)

In [35]:
for cell_type in ['HepG2', 'K562', 'SK-N-SH']:
    x = merged_df[f'{cell_type}_x']
    y = merged_df[f'{cell_type}_y']
    print(pearson(x, y))

(0.9980997662291563, 0.0)
(0.9980848663338745, 0.0)
(0.9981098008135579, 0.0)
